# Pickle exploration notebook

in thsi notebook there are some script in order to explore the different pickle files created for each person after the 2 models (original and optimized)

In [ ]:
import os
import pickle
import numpy as np
import pandas as pd

In [ ]:
subj_name = '410' # particpiant id change it to explote other participants
minutes = 5
personal_ID = xxxxxx # change xxxxxx with your personal ID

In [ ]:
BASE_PATH = (
    f"/run/user/{personal_ID}/gvfs/smb-share:server=ait-pdfs.win.dtu.dk,share=department/Man/"
    f"Public/4233-81647-eMOTIONAL-Cities/5 Data/ECDTU/E4/completed/{subj_name}/Data/"
)

# test_results_thrX.pkl
# marta_test_results_thrX.pkl


In [ ]:
def load_results(prefix):
    """
    prefix = 'test_results' ή 'marta_test_results'
    """
    results = {}

    for fname in os.listdir(BASE_PATH):
        if fname.startswith(prefix) and fname.endswith(".pkl"):
            thr = fname.replace(prefix + "_thr", "").replace(".pkl", "")
            thr = float(thr.replace("_", "."))

            with open(os.path.join(BASE_PATH, fname), "rb") as f:
                results[thr] = pickle.load(f)

    return results


test_results = load_results("test_results")
marta_results = load_results("marta_test_results")

print("Loaded thresholds (TEST): ", sorted(test_results.keys()))
print("Loaded thresholds (MARTA):", sorted(marta_results.keys()))


In [ ]:
rows = []

common_thresholds = sorted(set(test_results) & set(marta_results))

for thr in common_thresholds:

    tr = test_results[thr]
    mr = marta_results[thr]

    rows.append({
        "threshold": thr,

        # ---- TEST ----
        "test_mean_acc": tr["mean_acc"],
        "test_mean_acc_stress": tr["mean_acc_stress"],
        "test_mean_sil": tr["mean_sil"],

        # ---- MARTA ----
        "marta_mean_acc": mr["mean_acc"],
        "marta_mean_acc_stress": mr["mean_acc_stress"],
        "marta_mean_sil": mr["mean_sil"],

        # ---- DIFFERENCES ----
        "Δ_acc": mr["mean_acc"] - tr["mean_acc"],
        "Δ_acc_stress": mr["mean_acc_stress"] - tr["mean_acc_stress"],
        "Δ_sil": mr["mean_sil"] - tr["mean_sil"],
    })

summary_df = pd.DataFrame(rows).sort_values("threshold").reset_index(drop=True)

print("\n================ THRESHOLD COMPARISON =================\n")
display(summary_df)


In [ ]:
long_rows = []

for thr in common_thresholds:

    for fr in test_results[thr]["fold_results"]:
        long_rows.append({
            "threshold": thr,
            "method": "TEST",
            "fold": fr["fold"],
            "acc": fr["acc"],
            "acc_stress": fr["acc_stress"],
            "sil": fr["sil"]
        })

    for fr in marta_results[thr]["fold_results"]:
        long_rows.append({
            "threshold": thr,
            "method": "MARTA",
            "fold": fr["fold"],
            "acc": fr["acc"],
            "acc_stress": fr["acc_stress"],
            "sil": fr["sil"]
        })

long_df = pd.DataFrame(long_rows)

print("\n================ FOLD-LEVEL DATA =================\n")
display(long_df.head())


In [ ]:
import pickle
import pandas as pd
from datetime import timedelta

personal_ID = 1036
subj_name = '339'

FILE = (
    f"/run/user/{personal_ID}/gvfs/smb-share:server=ait-pdfs.win.dtu.dk,share=department/Man/"
    f"Public/4233-81647-eMOTIONAL-Cities/5 Data/ECDTU/E4/completed/{subj_name}/Data/"
    f"marta_test_results_thr10_5.pkl"   # ← change threshold as needed
)

with open(FILE, "rb") as f:
    data = pickle.load(f)


In [ ]:
fold0 = data["fold_results"][0]

time_sec = fold0["time_sec"]          # seconds since t0
ema_idx  = fold0["ema_idx"]
t0_dt    = fold0["t0_datetime"]        # pandas Timestamp


In [ ]:
print(fold0.keys())

In [ ]:
timestamps = [
    t0_dt + timedelta(seconds=int(s))
    for s in time_sec
]


In [ ]:
print("ema_idx:", len(ema_idx))
print("time_sec:", len(time_sec))
print("prob_stress:", len(fold0["prob_stress_ema"]))
print("label_true:", len(fold0["labels_true_ema"]))
print("label_pred:", len(fold0["labels_pred_ema"]))


In [ ]:
# ALIGN TO EMA TIMELINE 

L = len(ema_idx)   # = 5672

ema_idx  = ema_idx[:L]
time_sec = time_sec[:L]

prob_s = fold0["prob_stress_ema"][:L]
y_true = fold0["labels_true_ema"][:L]
y_pred = fold0["labels_pred_ema"][:L]


In [ ]:
from datetime import timedelta

t0_dt = fold0["t0_datetime"]  

timestamps = [
    t0_dt + timedelta(seconds=int(s))
    for s in time_sec
]


In [ ]:
# allignment with EMA timeline MATTEO FOR ADELFOS 
L = min(
    len(ema_idx),
    len(time_sec),
    len(fold0["prob_stress_ema"]),
    len(fold0["labels_true_ema"]),
    len(fold0["labels_pred_ema"]),
)

ema_idx  = ema_idx[:L]
time_sec = time_sec[:L]

prob_s = fold0["prob_stress_ema"][:L]
y_true = fold0["labels_true_ema"][:L]
y_pred = fold0["labels_pred_ema"][:L]

t0_dt = fold0["t0_datetime"]

timestamps = [
    t0_dt + timedelta(seconds=int(s))
    for s in time_sec
]

timeline_df = pd.DataFrame({
    "ema_idx": ema_idx,
    "time_sec": time_sec,
    "datetime": timestamps,
    "prob_stress": prob_s,
    "label_true": y_true,
    "label_pred": y_pred,
})



In [ ]:
print("LENGTH CHECK")
print("ema_idx        :", len(ema_idx))
print("time_sec       :", len(time_sec))
print("prob_stress    :", len(prob_s))
print("label_true     :", len(y_true))
print("label_pred     :", len(y_pred))
print("timestamps     :", len(timestamps))


In [ ]:
print("\nFIRST 5 ROWS CHECK")
for i in range(5):
    print(
        f"EMA {ema_idx[i]:4d} | "
        f"sec {time_sec[i]:6d} | "
        f"time {timestamps[i]} | "
        f"P(stress) {prob_s[i]:.3f} | "
        f"true {y_true[i]} | pred {y_pred[i]}"
    )


In [ ]:
# COUNTS: TRUE vs PRED (absolute)
counts_df = pd.DataFrame({
    "true": timeline_df["label_true"].value_counts(),
    "pred": timeline_df["label_pred"].value_counts()
}).fillna(0).astype(int).sort_index()

print("=== COUNTS (absolute) ===")
display(counts_df)


# COUNTS: TRUE vs PRED (percentages)
counts_pct_df = pd.DataFrame({
    "true_%": timeline_df["label_true"].value_counts(normalize=True),
    "pred_%": timeline_df["label_pred"].value_counts(normalize=True)
}).sort_index()

print("=== COUNTS (percentages) ===")
display(counts_pct_df)


# CONFUSION MATRIX (1 = no stress, 2 = stress)
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(
    timeline_df["label_true"],
    timeline_df["label_pred"],
    labels=[1, 2]
)

cm_df = pd.DataFrame(
    cm,
    index=["true_no_stress (1)", "true_stress (2)"],
    columns=["pred_no_stress (1)", "pred_stress (2)"]
)

print("=== CONFUSION MATRIX ===")
display(cm_df)


# STRESS RECALL
if cm[1].sum() > 0:
    stress_recall = cm[1, 1] / cm[1].sum()
else:
    stress_recall = None

print("Stress recall (true stress predicted as stress):", stress_recall)


# QUICK SANITY CHECK (time ordered)
print("=== FIRST 20 TIMEPOINTS ===")
display(
    timeline_df
    .sort_values("datetime")[["datetime", "label_true", "label_pred", "prob_stress"]]
    .head(20)
)

In [ ]:
# fuction to read the file the right way 
def parse_threshold(fname, prefix):
    thr = fname.replace(prefix + "_thr", "").replace(".pkl", "")
    return float(thr.replace("_", "."))

def load_results(prefix, base_path):
    results = {}

    for fname in os.listdir(base_path):
        if fname.startswith(prefix + "_thr") and fname.endswith(".pkl"):
            thr = parse_threshold(fname, prefix)

            with open(os.path.join(base_path, fname), "rb") as f:
                results[thr] = pickle.load(f)

    return results


In [ ]:


all_rows = []

subjects = ['487', '486', '483', '482']  # put subjects to analyze

for subj_name in subjects:

    BASE_PATH = (
        f"/run/user/{personal_ID}/gvfs/smb-share:server=ait-pdfs.win.dtu.dk,share=department/Man/"
        f"Public/4233-81647-eMOTIONAL-Cities/5 Data/ECDTU/E4/completed/{subj_name}/Data/"
    )

    test_results  = load_results("test_results", BASE_PATH)
    marta_results = load_results("marta_test_results", BASE_PATH)

    # thresholds that exist for BOTH models for THIS person
    common_thresholds = sorted(set(test_results) & set(marta_results))

    for thr in common_thresholds:

        tr = test_results[thr]
        mr = marta_results[thr]

        all_rows.append({
            "subject": subj_name,
            "threshold": thr,
            "method": "TEST",
            "acc": tr["mean_acc"],
            "acc_stress": tr["mean_acc_stress"],
            "sil": tr["mean_sil"],
        })

        all_rows.append({
            "subject": subj_name,
            "threshold": thr,
            "method": "MARTA",
            "acc": mr["mean_acc"],
            "acc_stress": mr["mean_acc_stress"],
            "sil": mr["mean_sil"],
        })

all_df = pd.DataFrame(all_rows)
display(all_df)


In [ ]:
# aggregation across subjects
summary = (
    all_df
    .groupby(["threshold", "method"])
    .agg(
        acc_mean=("acc", "mean"),
        acc_std=("acc", "std"),
        acc_stress_mean=("acc_stress", "mean"),
        acc_stress_std=("acc_stress", "std"),
        sil_mean=("sil", "mean"),
        sil_std=("sil", "std"),
        n_subjects=("subject", "nunique"),
    )
    .reset_index()
)

display(summary)


In [ ]:
import matplotlib.pyplot as plt

for method in ["TEST", "MARTA"]:
    tmp = summary[summary.method == method]

    plt.errorbar(
        tmp.threshold,
        tmp.acc_stress_mean,
        yerr=tmp.acc_stress_std,
        marker="o",
        label=method
    )

plt.xlabel("Threshold")
plt.ylabel("Mean stress accuracy")
plt.legend()
plt.grid(True)
plt.show()


----------------------------

In [ ]:
import pickle
import os
import numpy as np

In [ ]:
threshold = 7.5 # change it to explore other thresholds for that specific participant
subj_name = '103' # particpiant id change it to explote other participants


BASE_PATH = (
    f"/run/user/{personal_ID}/gvfs/smb-share:server=ait-pdfs.win.dtu.dk,share=department/Man/"
    f"Public/4233-81647-eMOTIONAL-Cities/5 Data/ECDTU/E4/completed/{subj_name}/Data/"
)

FILE = os.path.join(
    BASE_PATH,
    f"marta_test_results_thr{str(threshold).replace('.', '_')}.pkl"
    # f"test_results_thr{str(threshold).replace('.', '_')}.pkl" # uncomment to use the optimized model
)

with open(FILE, "rb") as f:
    data = pickle.load(f)

type(data)

In [ ]:
data.keys()

In [ ]:
data['hyperparameters']

In [ ]:
len(data["fold_results"])
data["fold_results"][0].keys()

In [ ]:
fold_results = data["fold_results"]
print(np.unique(fold_results[0]["labels_pred_ema"]))
print(fold_results[0]["U_test"].shape)


In [ ]:
import numpy as np

fr = data["fold_results"][0]

print("unique TRUE EMA:", np.unique(fr["labels_true_ema"], return_counts=True))
print("unique PRED EMA:", np.unique(fr["labels_pred_ema"], return_counts=True))

# sanity: che range hanno le prob
ps = np.array(fr["prob_stress_ema"])
print("prob_stress_ema min/max:", ps.min(), ps.max())


In [ ]:
true = np.array(fr["labels_true_ema"])
ps   = np.array(fr["prob_stress_ema"])

for lab in np.unique(true):
    m = ps[true == lab].mean()
    print("mean prob_stress_ema when true==", lab, ":", m)


In [ ]:
w = data["fold_results"][0]["ae_state_dict"]

print(type(w))
print(len(w))

print(list(w.keys())[:10])

for k, v in w.items():
    print(k, v.shape)
    break

w0 = data["fold_results"][0]["ae_state_dict"]
w1 = data["fold_results"][1]["ae_state_dict"]


In [ ]:
import os

BASE_PATH = f"/run/user/{personal_ID}/gvfs/smb-share:server=ait-pdfs.win.dtu.dk,share=department/Man/Public/4233-81647-eMOTIONAL-Cities/5 Data/ECDTU/E4/completed/"

missing_people = {}

for name in os.listdir(BASE_PATH):
    person_path = os.path.join(BASE_PATH, name)
    if not os.path.isdir(person_path):
        continue
    if not name.isdigit():
        continue

    files = os.listdir(os.path.join(person_path, "Data"))

    has_test = any(f.startswith("test_results_") for f in files)
    has_marta = any(f.startswith("marta_test_results_") for f in files)

    if not (has_test and has_marta):
        reason = []
        if not has_test:
            reason.append("missing test_results")
        if not has_marta:
            reason.append("missing marta_test_results")
        missing_people[name] = reason


for k in sorted(missing_people):
    print(k, "->", ", ".join(missing_people[k]))



In [ ]:
print(list(missing_people.keys()))